In [1]:
import pandas as pd
import os
import pickle
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.utils import shuffle
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [2]:
epi = os.listdir('static')
epi = [x for x in epi if 'all' in x]
for i,filn in enumerate(epi):
    if i ==0:
        df = pd.read_json('static/'+filn)
    else:
        df = df.append(pd.read_json('static/'+filn),ignore_index = True)

df['Number of Ratings'] = df['Number of Ratings'].apply(lambda x: int(str(x).replace('Ratings','')))
def return_length(x):
    if type(x) != type([]) and type(x) != type(1.0):
        x = x.split()
        x = len(x)
        return x
    elif type(x) == type([]):
        return len(' '.join(x))
    else:
        return None
def return_join(x):
    if type(x) == type([]):
        return ' '.join(x).lower()
    else:
        return x.lower()
def return_length_list(x):
    if type(x) ==str:
        return 1
    elif type(x) == list:
        return len(x)
    else:
        return None
    
df['lname'] = df['Recipe Name'].apply(return_length)
df['ldesc'] = df['Description'].apply(return_length)
df['lingredients'] = df['Ingredients'].apply(return_length_list)
df['linstructions'] = df.Instructions.apply(return_length_list)
df = df.dropna()
df.Ingredients = df.Ingredients.apply(lambda x: return_join(x))
df.Description = df.Description.apply(lambda x: return_join(x))
df['Recipe Name'] = df['Recipe Name'].apply(lambda x: return_join(x))
df.Instructions = df.Instructions.apply(lambda x: return_join(x))

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 341023 entries, 0 to 341022
Data columns (total 11 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   Recipe Name              341023 non-null  object
 1   Description              341023 non-null  object
 2   Ingredients              341023 non-null  object
 3   Original Recipe Website  341023 non-null  object
 4   Instructions             341023 non-null  object
 5   Number of Ratings        341023 non-null  int64 
 6   Publish Date             341023 non-null  int64 
 7   lname                    341023 non-null  int64 
 8   ldesc                    341023 non-null  int64 
 9   lingredients             341023 non-null  int64 
 10  linstructions            341023 non-null  int64 
dtypes: int64(6), object(5)
memory usage: 31.2+ MB


In [4]:
#df = df[df['Original Recipe Website'].str.contains('allrecipes') == True]

In [5]:
df['rate'] = np.where(df['Number of Ratings'] >= 50,1,0 )
df['words'] = df['Recipe Name'].str.cat(df['Description'],sep=" ").str.cat(df['Ingredients'],sep=" ").str.cat(df['Instructions'],sep=" ")

In [6]:
df.describe()

,Number of Ratings,Publish Date,lname,ldesc,lingredients,linstructions,rate
count,341023.000000,341023.000000,341023.000000,341023.000000,341023.000000,341023.000000,341023.000000
mean,10.464925,3727.436343,3.849236,22.417239,10.006096,1.763887,0.032200
std,127.716364,1462.131117,1.920262,68.141126,4.924848,1.907490,0.176532
min,0.000000,24.000000,0.000000,0.000000,1.000000,1.000000,0.000000
25%,0.000000,2758.000000,3.000000,0.000000,7.000000,1.000000,0.000000
50%,0.000000,3707.000000,3.000000,8.000000,9.000000,1.000000,0.000000
75%,0.000000,4588.000000,5.000000,27.000000,12.000000,1.000000,0.000000
max,16861.000000,8626.000000,61.000000,3336.000000,148.000000,89.000000,1.000000


In [7]:
dfadd = df[df.rate == 1]

In [8]:
dfadd.describe()

,Number of Ratings,Publish Date,lname,ldesc,lingredients,linstructions,rate
count,10981.000000,10981.000000,10981.000000,10981.000000,10981.000000,10981.000000,10981.0
mean,271.548857,5752.128130,3.799836,28.619160,9.387670,3.541663,1.0
std,659.601172,1961.627981,1.526512,43.325715,3.939949,1.782995,0.0
min,50.000000,119.000000,1.000000,0.000000,1.000000,1.000000,1.0
25%,71.000000,4264.000000,3.000000,15.000000,7.000000,2.000000,1.0
50%,111.000000,6014.000000,3.000000,23.000000,9.000000,3.000000,1.0
75%,230.000000,7656.000000,4.000000,36.000000,12.000000,4.000000,1.0
max,16861.000000,8625.000000,20.000000,2026.000000,39.000000,25.000000,1.0


In [9]:
for i in range(30):
    df = df.append(dfadd,ignore_index = True)

In [10]:
df.describe()

,Number of Ratings,Publish Date,lname,ldesc,lingredients,linstructions,rate
count,670453.000000,670453.000000,670453.000000,670453.000000,670453.000000,670453.000000,670453.000000
mean,138.749651,4722.277466,3.824963,25.464580,9.702230,2.637405,0.507733
std,488.967373,2000.622115,1.738120,57.389957,4.478731,2.050002,0.499941
min,0.000000,24.000000,0.000000,0.000000,1.000000,1.000000,0.000000
25%,0.000000,3151.000000,3.000000,5.000000,7.000000,1.000000,0.000000
50%,50.000000,4467.000000,3.000000,18.000000,9.000000,2.000000,1.000000
75%,112.000000,6254.000000,5.000000,33.000000,12.000000,4.000000,1.000000
max,16861.000000,8626.000000,61.000000,3336.000000,148.000000,89.000000,1.000000


In [11]:
columns =df.columns
columns = columns[0:-1]
columns

Index(['Recipe Name', 'Description', 'Ingredients', 'Original Recipe Website',
       'Instructions', 'Number of Ratings', 'Publish Date', 'lname', 'ldesc',
       'lingredients', 'linstructions', 'rate'],
      dtype='object')

In [12]:
X = df[df['Publish Date'] > 1000][['Recipe Name','Description','Ingredients','Instructions', 'lname', 'ldesc','lingredients', 'linstructions']]
y = df[df['Publish Date'] > 1000]['rate']

In [13]:
#X = df[df['Publish Date'] > 1000][['words','lname', 'ldesc','lingredients', 'linstructions']]
#y = df[df['Publish Date'] > 1000]['Number of Ratings']

In [14]:
#X2 = df[(df['Publish Date'] < 1000) & (df['Publish Date'] > 500)][['words', 'lname', 'ldesc','lingredients', 'linstructions']]
#y2 = df[(df['Publish Date'] < 1000) & (df['Publish Date'] > 500)]['Number of Ratings']

In [15]:
X2 = df[(df['Publish Date'] < 1000) & (df['Publish Date'] > 500)][['Recipe Name','Description','Ingredients','Instructions', 'lname', 'ldesc','lingredients', 'linstructions']]
y2 = df[(df['Publish Date'] < 1000) & (df['Publish Date'] > 500)]['rate']

In [16]:
df.describe()

,Number of Ratings,Publish Date,lname,ldesc,lingredients,linstructions,rate
count,670453.000000,670453.000000,670453.000000,670453.000000,670453.000000,670453.000000,670453.000000
mean,138.749651,4722.277466,3.824963,25.464580,9.702230,2.637405,0.507733
std,488.967373,2000.622115,1.738120,57.389957,4.478731,2.050002,0.499941
min,0.000000,24.000000,0.000000,0.000000,1.000000,1.000000,0.000000
25%,0.000000,3151.000000,3.000000,5.000000,7.000000,1.000000,0.000000
50%,50.000000,4467.000000,3.000000,18.000000,9.000000,2.000000,1.000000
75%,112.000000,6254.000000,5.000000,33.000000,12.000000,4.000000,1.000000
max,16861.000000,8626.000000,61.000000,3336.000000,148.000000,89.000000,1.000000


In [32]:
tfidf1 = TfidfVectorizer(min_df =2, max_features = 100)
tfidf2 = TfidfVectorizer(min_df = 2,max_features = 100)
tfidf3 = TfidfVectorizer(min_df = 2,max_features = 100)
tfidf4 = TfidfVectorizer(min_df = 2,max_features = 100)
model = Pipeline([('cst',ColumnTransformer([("tfidf1", tfidf1,'Recipe Name'),
                                            ("tfidf3",tfidf3,'Description'),
                                            ("tfidf2", tfidf2,'Ingredients'),
                                            ("tfidf4", tfidf4,'Instructions')
                                          ]
                                           ,remainder="passthrough"
                                          )),
                    ('lr', GridSearchCV(RandomForestClassifier(criterion='entropy'),
                            {'n_estimators':[100,500,1000]},
                           cv = 5,
                           n_jobs=4))
                 ])

In [ ]:
model.fit(X,y)

In [62]:
model['lr'].best_params_

{'C': 1000000000000000.0}

In [ ]:
import pickle
pickle.dump(model,open('model_10.p','wb'))

In [ ]:
model = pickle.load(open('model_1.p','rb'))

In [29]:
model.score(X2,y2)

0.8004479019330505

In [31]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
y_pred = model.predict(X2)
print(classification_report(y2, y_pred, digits=4))
max(y)

              precision    recall  f1-score   support

           0     0.8030    0.9926    0.8878      6748
           1     0.6503    0.0536    0.0990      1736

    accuracy                         0.8004      8484
   macro avg     0.7267    0.5231    0.4934      8484
weighted avg     0.7718    0.8004    0.7264      8484



1

In [ ]:
epi = os.listdir('static')
epi = [x for x in epi if 'all' in x]
for i,filn in enumerate(epi):
    if i ==0:
        df3 = pd.read_json('static/'+filn)
    else:
        df3 = df3.append(pd.read_json('static/'+filn),ignore_index = True)
df3['Number of Ratings'] = df3['Number of Ratings'].apply(lambda x: int(str(x).replace('Ratings','')))
df3 = df3[df3['Publish Date']<=365]
df2 = df3.copy()
df3['lname'] = df3['Recipe Name'].apply(return_length)
df3['ldesc'] = df3['Description'].apply(return_length)
df3['lingredients'] = df3['Ingredients'].apply(len)
df3['linstructions'] = df3.Instructions.apply(len)
df3.Instructions = df3.Instructions.apply(lambda x: return_join(x))
df3.Ingredients = df3.Ingredients.apply(lambda x: return_join(x))
df3.Description = df3.Description.apply(lambda x: return_join(x))
df3['Recipe Name'] = df3['Recipe Name'].apply(lambda x: return_join(x))
df2.Instructions = df2.Instructions.apply(lambda x: return_join_web(x))
df2.Ingredients = df2.Ingredients.apply(lambda x: return_join_web(x))
df2.Description = df2.Description.apply(lambda x: return_join_web(x))
df2['Recipe Name'] = df2['Recipe Name'].apply(lambda x: return_join_web(x))
df3 = df3.dropna()
df2= df2.dropna()
df3['words'] = df3['Recipe Name'].str.cat(df3['Description'],sep=" ").str.cat(df3['Ingredients'],sep=" ").str.cat(df3['Instructions'],sep=" ")
X = df3[df3['Publish Date'] <= 365][['Recipe Name','Description','Ingredients','Instructions', 'lname', 'ldesc','lingredients', 'linstructions']]
df2 = df2.drop(['Publish Date'],axis = 1)
print(len(df2),len(X))
ypred = model.predict(X)
df2['Recommended'] = ypred
df2 = df2.drop_duplicates(subset=['Recipe Name','Description','Ingredients'], keep='last')
df2.to_json('static/web.json',orient='records')

In [ ]:
def return_join_web(x):
    temp = []
    count = 1
    if type(x) == type([]):
        for i in x:
            temp.append('[' + str(count)+ '] '+i)
            count+=1
        return '<br/>'.join(temp)
    else:
        return x

In [ ]:
df2